### Implements algorithm 1 in the parti-game [paper](https://link.springer.com/content/pdf/10.1007/BF00993591.pdf)

Learn a controller from a start region to a goal region on a continuous space; Four increasingly effective algorithms to partition discrete state spaces. Algorithm 1 and 2 are non-learning; 3 and 4 learn, hence explore, the world while planning a route to the goal. Here, I implement algorithm 1

In [106]:
import sys
import numpy as np
from os.path import abspath, join
sys.path.append(abspath(join('..')))
sys.path.append(abspath(join('..', 'DynamicalSystems')))
sys.path.append(abspath(join('..', 'grids')))
sys.path.append(abspath(join('..', 'utils')))
sys.path.append(abspath(join('..', 'Visualization')))
sys.path.append(abspath(join('..')))

from Grids import *
import matplotlib.pyplot as plt
from gen_utils import *
from BoundaryCondition import *
from DynamicalSystems import *
from Visualization import Visualizer
from ValueFuncs import proj
%matplotlib inline

In [107]:
#Given N cells, construct an MDT with N discrete states 1, ..., N.

N = 6

# make a 6 cell grid
gridIn=expand(np.array((0, 1, 0, 1, 0, 1)), 1)
gridOut =expand(np.array((1, 2, 1, 2, 1, 2)), 1)
num_points = 30
N = num_points*ones(6,1).astype(np.int64)

g = createGrid(gridIn, gridOut, N, process=True, low_mem=False);

# print(f'len(g.xs), g.xs[0].shape {len(g.xs), g.xs[0].shape} g.N {g.N.shape}')
dims = [[0, 4], [1, 5], [2, 3]]

bounds = [np.linspace(0, 1, 3), np.linspace(1, 2, 4), np.linspace(2, 3, 3), np.linspace(3, 4, 4)]
        
if g.dim > 1:
    bounds_grid = np.meshgrid(*bounds, sparse=False);
else:
    bounds_grid = np.meshgrid(bounds)[0]

Grid dimension > 5, may be dangerously large


In [3]:
N = 4

# make a 6 cell grid
gridIn=expand(np.array((0, 0)), 1)
gridOut =expand(np.array((1, 1)), 1)
num_points = 30
N = num_points*ones(2,1).astype(np.int64)

g = createGrid(gridIn, gridOut, N, process=True, low_mem=False);

bounds = [[0, 0.5, 1], [0, 0.25, 0.75, 1]]
padding = np.array(([[0.1, 0.2]])).T

if g.dim > 1:
    bounds_grid = np.meshgrid(*bounds, sparse=False, indexing='ij');
else:
    bounds_grid = np.meshgrid(bounds)[0]

bds_grd = bounds_grid[0]
bds_grd_shp = np.array(bds_grd.shape)

gs = bds_grd_shp-(bds_grd_shp>1)
gs = np.empty(gs)

In [21]:
a = Bundle({})
a.x = 1
a

In [22]:
a.x

1

In [15]:
k = Bundle(dict(a = 0))

In [16]:
type(k), k.dtype

(__main__.Bundle,
 <bound method Bundle.dtype of <__main__.Bundle object at 0x7f9bcf92ca90>>)

In [18]:
A = np.arange(12).reshape(4, 3)
np.all(A)

True

In [36]:
x = np.zeros((2,1))
x.fill(np.nan)
print(x)

[[nan]
 [nan]]


In [40]:
import casadi

In [55]:
dubins_default_params = dict(
                            wRange=[-1, 1],
                            speed=5,
                            dRange=zeros(3,2),
                            dims=np.arange(3),
                            end=None,
                            nu = 1,
                            nd = 3,
                            x = None,
                            nx=3,
                            u = None,
                            xhist=None,
                            uhist=None,
                            pdim=None,
                            vdim=None,
                            hdim=None,
                            hpxpy=None,
                            hpxpyhist=None,
                            hvxvy=None,
                            hvxvyhist=None,
                            hpv = cell(2,1),
                            hpvhist = cell(2,1),
                            data = None
                            )
dubins_default_params['speed']

5

In [56]:
new_params = {'speed': 1}
dubins_default_params.update(new_params)
dubins_default_params['speed']

1

In [42]:
from casadi import vertcat

In [101]:
data = np.random.rand(41, 41, 41)

for i in range(3):
    to_app = np.expand_dims(data[..., i], i)
    data = np.concatenate((data, to_app), i)
    print(data.shape)

(42, 41, 41)
(42, 42, 41)
(42, 42, 42)


In [103]:
data = np.arange(12).reshape(3,2,2)
print(data)

[[[ 0  1]
  [ 2  3]]

 [[ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]]]


In [104]:
data[1, ...]

array([[4, 5],
       [6, 7]])

In [90]:
max(_[0])

3

In [105]:
x= np.zeros((3, 1))
x.fill(True)
x

array([[1.],
       [1.],
       [1.]])